In [133]:
import numpy as np
import pandas as pd
import scipy
import nltk
import re
from nltk.tokenize import word_tokenize
from pymystem3 import Mystem
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
!pip install pymystem3

In [151]:
data = pd.read_csv('/kaggle/input/russian-language-toxic-comments/labeled.csv')
data.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [152]:
data['toxic'] = data['toxic'].astype(int)
data = data.replace('\n',' ', regex=True)
data = data.replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)
data['comment'] = data['comment'].str.lower()
data = data.replace('[^а-яА-я]', ' ', regex=True)

In [153]:
data.head()

,comment,toxic
0,верблюдов то за что дебилы бл,1
1,хохлы это отдушина затюканого россиянина мол...,1
2,собаке собачья смерть,1
3,страницу обнови дебил это тоже не оскорблени...,1
4,тебя не убедил страничный пдф в том что скр...,1


In [137]:
mystem = Mystem()

def lemmat(comment):
    word_list = word_tokenize(comment)
    lemmas = [mystem.lemmatize(w)[0] for w in word_list if not w.lower() in stop_list]
    return lemmas

for i in tqdm(range(len(data))):
    data['comment'][i] = lemmat(data["comment"][i])

  0%|          | 0/14412 [00:00<?, ?it/s]/tmp/ipykernel_27/4109446096.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['comment'][i] = lemmat(data["comment"][i])
100%|██████████| 14412/14412 [00:54<00:00, 265.34it/s]


In [138]:
data['comment'] = data['comment'].apply(lambda x: ', '.join(x))

In [139]:
nltk.download('punkt')
data['tokenized_comment'] = data['comment'].apply(nltk.word_tokenize)
data.head()

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,comment,toxic,tokenized_comment
0,"верблюд, ?, дебил, ,\n, бл, ...",1,"[верблюд, ,, ?, ,, дебил, ,, ,, ,, бл, ,, ...]"
1,"хохол, ,\n, это, отдушина, затюканый, россияни...",1,"[хохол, ,, ,, ,, это, ,, отдушина, ,, затюканы..."
2,"собака, -\n, собачий, смерть",1,"[собака, ,, -, ,, собачий, ,, смерть]"
3,"страница, обновлять, ,\n, дебил, ., это, оскор...",1,"[страница, ,, обновлять, ,, ,, ,, дебил, ,, .,..."
4,"убеждать, 6, пдф, ,\n, скрипалый, отравлять, р...",1,"[убеждать, ,, 6, ,, пдф, ,, ,, ,, скрипалый, ,..."


In [140]:
tokenized_comments = data['tokenized_comment'].apply(lambda x: ' '.join(x))
tfidf_vectors = vectorizer.fit_transform(tokenized_comments)

In [141]:
X = tfidf_vectors
y = data['toxic']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [142]:
N = 14412
l2_coef = 0.5
alpha = 2
beta = 2
batch_size = int(N/20)

clf = LogisticRegression(C=1/(2*l2_coef), solver='sag')
clf.fit(X_train, y_train)

y_pred_val = clf.predict(X_val)
y_pred_test = clf.predict(X_test)

threshold = 0.5

print("val acc:", balanced_accuracy_score(y_val, y_pred_val))
print("test acc:", balanced_accuracy_score(y_test, y_pred_test))

val acc: 0.779036482404578
test acc: 0.7861553202119407


Все слова с весами:

In [149]:
weights = clf.coef_[0]
word_weights = {word: weights[i] for word, i in vectorizer.vocabulary_.items()}
word_weight_pairs = [(word, weight) for word, weight in word_weights.items()]
df = pd.DataFrame(word_weight_pairs, columns=['word', 'weight'])
print(df.head(10))

        word    weight
0    верблюд  0.282320
1      дебил  3.756881
2         бл  0.611891
3      хохол  5.117917
4        это -1.296658
5   отдушина  0.156241
6  затюканый  0.000000
7  россиянин  0.626020
8        мол  0.428603
9        вон  0.692230


Только плохие:

In [150]:
filtered_df = df[df['weight'] > 1]
print(filtered_df.head(10))

          word    weight
1        дебил  3.756881
3        хохол  5.117917
10      хохлов  3.791348
25      писать  1.479857
27        твой  3.520533
38      ватник  1.629258
48       тупой  3.984838
84       ебать  2.947675
86       шизик  1.743394
87  обсираться  1.511760
